In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import random
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from Faster_RCNN.Train import Train_Faster_RCNN
from Faster_RCNN.Test import Test_Faster_RCNN
from Faster_RCNN.Valid import Valid_Faster_RCNN
from Faster_RCNN.Toolbox import check_dataset

seed= 1
random.seed(seed)
np.random.seed(seed)

# Prepare Data

In [ ]:
import json

all_dataset= []

with open('./Data/train_1.json', 'r', encoding="utf-8") as f:
    label= json.load(f)  
for i in tqdm(range(len(label['images']))):
    data= {}
    data['image_path']= './Data/train_img_1/' + label['images'][i]['file_name']
    data['bbox']= []
    data['label']= []
    id= label['images'][i]['id']
    
    for j in range(len(label['annotations'])):
        if id==label['annotations'][j]['image_id']:
            box= label['annotations'][j]['bbox']
            box[2]+= box[0]
            box[3]+= box[1]
            data['bbox'].append(box)
            data['label'].append(label['annotations'][j]['category_id'])
    all_dataset.append(data)
    
# with open('./Data/train_2.json', 'r', encoding="utf-8") as f:
#     label= json.load(f)  
# for i in tqdm(range(len(label['images']))):
#     data= {}
#     data['image_path']= './Data/train_img_2/' + label['images'][i]['file_name']
#     data['bbox']= []
#     data['label']= []
#     id= label['images'][i]['id']
    
#     for j in range(len(label['annotations'])):
#         if id==label['annotations'][j]['image_id']:
#             box= label['annotations'][j]['bbox']
#             box[2]+= box[0]
#             box[3]+= box[1]
#             data['bbox'].append(box)
#             data['label'].append(label['annotations'][j]['category_id'])
#     all_dataset.append(data)


# PL

In [ ]:
conf_thr= 0.3
df= pd.read_csv('Data/submission_0.475_1234+5.csv')
df= df[df['confidence']>conf_thr]

all_img_name= list(set(df['image_filename']))
for name in tqdm(all_img_name):
    data= df[df['image_filename']==name]
    bbox= data[['x', 'y', 'w', 'h']].values
    for i in range(len(bbox)):
        bbox[i][2]+= bbox[i][0]
        bbox[i][3]+= bbox[i][1]
    label= data['label_id']
    
    data= {}
    data['image_path']= f'Data/test_img/{name}'
    data['bbox']= np.array(bbox)
    data['label']= np.array(label)
    all_dataset.append(data)
    
all_dataset= check_dataset(all_dataset[:])
all_dataset[:3]

# 過濾標籤類別

In [ ]:
keep_classes= [1,2,3,4,5]

drop_sample= []
for i, data in enumerate(tqdm(all_dataset)):
    
    bbox= data['bbox']
    label= data['label']
    
    drop_indx= []
    for j in range(len(label)):
        if label[j] not in keep_classes:
            drop_indx.append(j)
            
    data['bbox']= np.delete(bbox, drop_indx, axis= 0)
    data['label']= np.delete(label, drop_indx, axis= 0)
    
    if len(data['bbox'])==0: drop_sample.append(i)

all_dataset= np.delete(all_dataset, drop_sample, axis= 0)

print('drop empty sample: {}'.format(len(drop_sample)))

# 將具有大量bbox的資料移至validation

In [ ]:
import matplotlib.pyplot as plt

drop_indx= []
for i, data in enumerate(tqdm(all_dataset)):
    
    #img= cv2.imread(data['image_path'])
    bbox= data['bbox']
    bbox= np.array(bbox).astype(np.int)

    if len(bbox)>150 and 'test_img' not in data['image_path']:
        drop_indx.append(i)
        continue
        print(len(bbox))
        for box in bbox:
            cv2.rectangle(img,
                          (box[0], box[1]),
                          (box[2], box[3]),
                          (255, 0, 0), 10)
        plt.imshow(img)
        plt.show()

vali_dataset= np.array(all_dataset)[drop_indx]
train_dataset= np.delete(all_dataset, drop_indx, axis= 0)
print('move {} sample to validation'.format(len(drop_indx)))

# Train

In [ ]:
# train_dataset sample
#For every data in train_dataset:
#    image_path: full path of image
#    bbox: [ [left_top_x, left_top_y, right_bottom_x, right_bottom_y], ... ]
#    label: [ 1, 2, 3, ... ]  # label numbering start from 1, every label correspond to one box in bbox
#
#example:
#  [
#     {'image_path': 'train/img/1615187460.03881810.jpg',
#      'bbox': array([[ 1.,  9., 31., 47.],
#         [33.,  9., 61., 47.],
#         [63.,  9., 93., 47.]], dtype=float32),
#      'label': [1, 1, 1]},
#  ]

In [ ]:
# class label start from 1, 0 is background class.

all_backbone=[
    'resnet18',
    'resnet34',
    'resnet50',
    'resnet101',
    'resnet152',
    'resnext50_32x4d',
    'resnext101_32x8d',
    'wide_resnet50_2',
    'wide_resnet101_2',
]

optimizer=[
    'adam',
    'sgd',
    'rmsprop',
]

In [ ]:
import torch
from torchvision.models.detection.transform import GeneralizedRCNNTransform

multi_scale= list(range(1280, 1792, 32))
best_score= 0
epoch= 1
for ep in range(epoch):
    print()
    print('epoch:', ep)

    Train_CFG= {
        'model_architecture': 'FasterRCNN',
        'epochs': 1,
        'model_backbone': 'resnet152',
        'pretrained': True,
        'img_size': 1536, 
        
        'lr': 1e-5,
        'optimizer': 'adam',
        'batch_size': 1,
        'anchor_ratio': 2,
        
        'save_best_model': False,
        'save_model_path_and_name': False,#'train_cv_model/faster_rcnn.pth',
        'load_model': False, #'test_cv_model/resnet152_1792_best/faster_rcnn_best.pth', 
        
        'mixup': True,
        'mosaic': False,
        'data_aug': True,
        
        'valid_train_dataset': False,
        'valid_size': 0,
        'device': 'gpu:0', # cpu or gpu:0 if you have
    }

    
    if ep!=0: 
#         choose_size= np.random.choice(multi_scale, 1)[0]
#         print(f'choose size: {choose_size}')
#         model.transform= GeneralizedRCNNTransform(min_size= choose_size,
#                                       max_size= 4096,
#                                       image_mean=[0.485, 0.456, 0.406],
#                                       image_std=[0.229, 0.224, 0.225])
        Train_CFG['load_model']= model

    # start training
    model= Train_Faster_RCNN(
                dataset= train_dataset[:],
                CFG= Train_CFG,
            )


    Valid_CFG= {
        'img_size': None,    # None為使用原始解析度
        'confidence': 0.001,
        'NMS_threshold': 0.5,
        'load_model': model,
        'device': 'gpu:0',
    }

    try:
        valid_IOU, valid_class_IOU, valid_MAP, valid_class_MAP= Valid_Faster_RCNN(
                                                                    dataset= vali_dataset,
                                                                    CFG= Valid_CFG,
                                                                )
        if valid_MAP > best_score:
            torch.save(model, 'train_cv_model/faster_rcnn_best.pth')
            best_score= valid_MAP
            print('model save at score:', best_score)
    except:
        print('model not converage')
        
    if ep!=0 and ep%5==0:
        torch.save(model, 'train_cv_model/faster_rcnn_{}.pth'.format(ep))